**Устанавливаем зависимости**

In [ ]:
!pip install qrcode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
!apt-get install -y poppler-utils libzbar0

In [ ]:
# Установка Poppler для pdf2image
!apt-get install poppler-utils
!pip install ultralytics numpy opencv-python-headless pillow easyocr pyzbar pdf2image gradio

In [ ]:
!pip install mlflow # было использовано для трекинга состояния модели

In [ ]:
import gradio as gr
from PIL import Image
import numpy as np
import cv2
import json
import io
import time
import os



# --- МОДЕЛИ И ЛОГИКА ---
MODELS_LOADED = False # По умолчанию False
YOLO_MODEL = None
OCR_READER = None

print("--- [DEBUG] Запуск загрузчика моделей ---")

try:
    print("--- [DEBUG] 1. Импорт библиотек...")
    from ultralytics import YOLO
    from pdf2image import convert_from_bytes
    from pyzbar.pyzbar import decode
    import easyocr
    import os # Добавлен импорт os, так как он используется ниже
    print("--- [DEBUG] 1. Импорт УСПЕШЕН.")

    print("--- [DEBUG] 2. Загрузка YOLO model (/content/best.pt)...")
    # Загружаем весы
    if not os.path.exists("/content/best.pt"):
        print("!!! КРИТИЧЕСКАЯ ОШИБКА: Файл /content/best.pt НЕ НАЙДЕН !!!")
        raise FileNotFoundError("/content/best.pt не найден")
    #@markdown **Введите снизу путь который указывает на весы(к примеру внутри YOLO("/content/best.pt")**
    YOLO_MODEL = YOLO("/content/best.pt")
    print("--- [DEBUG] 2. YOLO model УСПЕШНО ЗАГРУЖЕНА.")

    print("--- [DEBUG] 3. Загрузка EasyOCR reader (ru, en)...")
    OCR_READER = easyocr.Reader(['ru', 'en'])
    print("--- [DEBUG] 3. EasyOCR reader УСПЕШНО ЗАГРУЖЕН.")

    MODELS_LOADED = True
    print("\n AI Models successfully loaded. (MODELS_LOADED = True)")
"""
ALL ML WORK
WAS COMPLETED BY RYSBEK ORAZYMBETOV
"""
except Exception as e:
    print("\n--- [DEBUG] !!! КРИТИЧЕСКАЯ ОШИБКА ПРИ ЗАГРУЗКЕ МОДЕЛЕЙ !!! ---")
    print(f"ОШИБКА: {e}")
    import traceback
    traceback.print_exc()
    print("--- [DEBUG] (MODELS_LOADED = False) ---")
# --- ГЛОБАЛЬНАЯ ФУНКЦИЯ ОБРАБОТКИ ПАЙПЛАЙНА (НА ВСЕ СТРАНИЦЫ) ---
"""
ALL ML WORK
WAS COMPLETED BY RYSBEK ORAZYMBETOV
"""
def process_document_pipeline(input_file_path):
    """
    Принимает ПУТЬ к файлу, обрабатывает ВСЕ страницы PDF (или 1 для JPG)
    и возвращает Галерею изображений и JSON-статистику.
    """
    print("\n--- [DEBUG] 1. Pipeline started (Multi-page) ---")

    if not MODELS_LOADED:
        print("--- [DEBUG] ERROR: Models not loaded! ---")
        error_json = {"status": "error", "message": "Модели не загружены!"}
        return None, json.dumps(error_json, indent=2, ensure_ascii=False)

    start_time = time.time()
    all_elements = [] # Собираем элементы со ВСЕХ страниц
    processed_pil_images = [] # Собираем PIL-изображения для Галереи
    global_element_id = 0 # Сквозной ID для элементов
    page_num = 0 # Инициализация для обработки ошибок вне цикла

    try: # Загрузка изображений и прочие прелести
        print(f"--- [DEBUG] 2. Loading file: {input_file_path} ---")

        if input_file_path.lower().endswith('.pdf'):
            with open(input_file_path, 'rb') as f:
                pdf_bytes = f.read()

            print("--- [DEBUG] 2a. Converting PDF (ALL PAGES)... ---")
            # Убираем first_page и last_page, чтобы получить ВСЕ страницы
            pil_images = convert_from_bytes(pdf_bytes)
        else:
            print("--- [DEBUG] 2a. Opening image (single page)... ---")
            pil_images = [Image.open(input_file_path)]

        print(f"--- [DEBUG] 3. File loaded. Total pages: {len(pil_images)} ---")

        # !!! ГЛАВНЫЙ ЦИКЛ ОБРАБОТКИ СТРАНИЦ !!!
        for page_num, page_image_pil in enumerate(pil_images):

            print(f"\n--- [DEBUG] Processing PAGE {page_num + 1} / {len(pil_images)} ---")
            # Конвертируем PIL в формат CV2 (numpy array)
            image_cv2 = cv2.cvtColor(np.array(page_image_pil), cv2.COLOR_RGB2BGR)
            img_height, img_width, _ = image_cv2.shape

            # 2. YOLO-SEG (ПОИСК)
            print(f"--- [DEBUG] 4. (Page {page_num+1}) Running YOLO predict... ---")
            results = YOLO_MODEL.predict(image_cv2, verbose=False)
            result = results[0]
            print(f"--- [DEBUG] 5. (Page {page_num+1}) YOLO found {len(result.boxes.cls)} objects. ---")

            # Копируем изображение для отрисовки масок
            output_image = image_cv2.copy()

            # 3. ЦИКЛ ОБРАБОТКИ ЭЛЕМЕНТОВ (на текущей странице)
            for i in range(len(result.boxes.cls)):
                class_id = int(result.boxes.cls[i])
                class_name = YOLO_MODEL.names[class_id]
                bbox = result.boxes.xyxy[i].cpu().numpy().astype(int)
                x1, y1, x2, y2 = bbox
                print(f"--- [DEBUG] 6. (Page {page_num+1}) Processing object {i+1}: {class_name} ---")

                # Обрезаем изображение
                cropped_image_cv2 = image_cv2[y1:y2, x1:x2]
                content = "Detected (No text)"

                # --- Отрисовка маски ---
                if result.masks and result.masks.data.shape[0] > i:
                    mask = result.masks.data[i].cpu().numpy()
                    mask = cv2.resize(mask, (img_width, img_height))
                    mask_3ch = np.zeros_like(output_image, dtype=np.uint8)
                    mask_3ch[mask > 0] = [0, 0, 255] # BGR: Красное выделение
                    cv2.addWeighted(output_image, 0.7, mask_3ch, 0.3, 0, output_image)
                    cv2.rectangle(output_image, (x1, y1), (x2, y2), (255, 0, 0), 2) # Синий бокс

                # --- Чтение содержимого ---
                if class_name == 'qrcode':
                    decoded_qr = decode(cropped_image_cv2)
                    content = decoded_qr[0].data.decode('utf-8') if decoded_qr else "Error: Not readable"
                elif class_name == 'stamp':
                    ocr_results = OCR_READER.readtext(cropped_image_cv2, detail=0)
                    content = " ".join(ocr_results) if ocr_results else "Error: No text detected"

                # Добавляем элемент в ОБЩИЙ JSON
                all_elements.append({
                    "id": global_element_id, # Используем сквозной ID
                    "page": page_num + 1, # !!! ДОБАВЛЯЕМ НОМЕР СТРАНИЦЫ
                    "type": class_name,
                    "bbox_px": [int(x) for x in [x1, y1, x2, y2]],
                    "content": content
                })
                global_element_id += 1 # Увеличиваем счетчик

            # Конвертируем финальное CV2 изображение (текущей страницы) обратно в PIL
            final_pil_page = Image.fromarray(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
            # Добавляем в список для Галереи
            processed_pil_images.append(final_pil_page)

        # --- Конец главного цикла ---

        end_time = time.time()
        print(f"--- [DEBUG] 7. All {len(pil_images)} pages processed. ---")

        # Финальная сборка JSON-статистики (теперь она общая)
        json_output = {
            "status": "success",
            "total_pages_processed": len(pil_images),
            "processing_time_s": round(end_time - start_time, 3),
            "total_elements_found": len(all_elements),
            "elements": all_elements # Здесь теперь ВСЕ элементы
        }

        print("--- [DEBUG] 8. Pipeline finished successfully! ---")

        # Возвращаем СПИСОК изображений и JSON
        return processed_pil_images, json.dumps(json_output, indent=2, ensure_ascii=False)

    except Exception as e:
        print(f"\n--- [DEBUG] !!! KERNEL CRASHED (during page {page_num+1}) !!! ---")
        print(f"--- [DEBUG] ERROR: {e} ---")
        import traceback
        traceback.print_exc()

        error_json = {
            "status": "error",
            "message": f"Критическая ошибка на странице {page_num+1}: {e}",
            "traceback": traceback.format_exc()
        }
        # Возвращаем (None для Галереи, JSON для Ошибки)
        return None, json.dumps(error_json, indent=2, ensure_ascii=False)
# --- НАСТРОЙКА И ЗАПУСК GRADIO ДЛЯ ДЕМО ---

# Настройка интерфейса
interface = gr.Interface(
    fn=process_document_pipeline,
    inputs=gr.File(label="Загрузите документ (PDF, JPG, PNG)"),
    outputs=[
        # !!! ИЗМЕНЕНО: gr.Image на gr.Gallery !!!
        gr.Gallery(label="Обработанные страницы (Галерея)",
                   # preview=True позволяет просматривать изображения в полном размере
                   preview=True),
        gr.JSON(label="JSON Статистика (со всех страниц)")
    ],
    title="Document AI Parser INNOVATEX(Хакатон)",
    description="Загрузите PDF или изображение. Демо обрабатывает ВСЕ страницы PDF. dm tg @fadeinvoid",
    allow_flagging='never'
)

# Запуск.
if __name__ == "__main__":
    interface.launch(share=True, debug=True)



--- [DEBUG] Запуск загрузчика моделей ---
--- [DEBUG] 1. Импорт библиотек...
--- [DEBUG] 1. Импорт УСПЕШЕН.
--- [DEBUG] 2. Загрузка YOLO model (/content/best.pt)...
--- [DEBUG] 2. YOLO model УСПЕШНО ЗАГРУЖЕНА.
--- [DEBUG] 3. Загрузка EasyOCR reader (ru, en)...
--- [DEBUG] 3. EasyOCR reader УСПЕШНО ЗАГРУЖЕН.

✅ AI Models successfully loaded. (MODELS_LOADED = True)


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cb6897cb2e144196a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",


--- [DEBUG] 1. Pipeline started (Multi-page) ---
--- [DEBUG] 2. Loading file: /tmp/gradio/2e292b0085aaeda2a805a69c3d70dbd07c4b2a17783794205655a4022612487c/АПЗ-.pdf ---
--- [DEBUG] 2a. Converting PDF (ALL PAGES)... ---
--- [DEBUG] 3. File loaded. Total pages: 9 ---

--- [DEBUG] Processing PAGE 1 / 9 ---
--- [DEBUG] 4. (Page 1) Running YOLO predict... ---
--- [DEBUG] 5. (Page 1) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 1) Processing object 1: qrcode ---

--- [DEBUG] Processing PAGE 2 / 9 ---
--- [DEBUG] 4. (Page 2) Running YOLO predict... ---
--- [DEBUG] 5. (Page 2) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 2) Processing object 1: qrcode ---

--- [DEBUG] Processing PAGE 3 / 9 ---
--- [DEBUG] 4. (Page 3) Running YOLO predict... ---
--- [DEBUG] 5. (Page 3) YOLO found 0 objects. ---

--- [DEBUG] Processing PAGE 4 / 9 ---
--- [DEBUG] 4. (Page 4) Running YOLO predict... ---
--- [DEBUG] 5. (Page 4) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 4) Processing object 1: qrcode ---

-

...


--- [DEBUG] 1. Pipeline started (Multi-page) ---
--- [DEBUG] 2. Loading file: /tmp/gradio/12cbc82ca2c57dc9750c76f87cf94cd4833e69658f66dd8f4f713358677575a1/Археология-2.pdf ---
--- [DEBUG] 2a. Converting PDF (ALL PAGES)... ---
--- [DEBUG] 3. File loaded. Total pages: 2 ---

--- [DEBUG] Processing PAGE 1 / 2 ---
--- [DEBUG] 4. (Page 1) Running YOLO predict... ---
--- [DEBUG] 5. (Page 1) YOLO found 0 objects. ---

--- [DEBUG] Processing PAGE 2 / 2 ---
--- [DEBUG] 4. (Page 2) Running YOLO predict... ---
--- [DEBUG] 5. (Page 2) YOLO found 2 objects. ---
--- [DEBUG] 6. (Page 2) Processing object 1: stamp ---
--- [DEBUG] 6. (Page 2) Processing object 2: stamp ---
--- [DEBUG] 7. All 2 pages processed. ---
--- [DEBUG] 8. Pipeline finished successfully! ---


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",


--- [DEBUG] 1. Pipeline started (Multi-page) ---
--- [DEBUG] 2. Loading file: /tmp/gradio/2e292b0085aaeda2a805a69c3d70dbd07c4b2a17783794205655a4022612487c/АПЗ-.pdf ---
--- [DEBUG] 2a. Converting PDF (ALL PAGES)... ---
--- [DEBUG] 3. File loaded. Total pages: 9 ---

--- [DEBUG] Processing PAGE 1 / 9 ---
--- [DEBUG] 4. (Page 1) Running YOLO predict... ---
--- [DEBUG] 5. (Page 1) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 1) Processing object 1: qrcode ---

--- [DEBUG] Processing PAGE 2 / 9 ---
--- [DEBUG] 4. (Page 2) Running YOLO predict... ---
--- [DEBUG] 5. (Page 2) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 2) Processing object 1: qrcode ---

--- [DEBUG] Processing PAGE 3 / 9 ---
--- [DEBUG] 4. (Page 3) Running YOLO predict... ---
--- [DEBUG] 5. (Page 3) YOLO found 0 objects. ---

--- [DEBUG] Processing PAGE 4 / 9 ---
--- [DEBUG] 4. (Page 4) Running YOLO predict... ---
--- [DEBUG] 5. (Page 4) YOLO found 1 objects. ---
--- [DEBUG] 6. (Page 4) Processing object 1: qrcode ---

-

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",


--- [DEBUG] 1. Pipeline started (Multi-page) ---
--- [DEBUG] 2. Loading file: /tmp/gradio/12cbc82ca2c57dc9750c76f87cf94cd4833e69658f66dd8f4f713358677575a1/Археология-2.pdf ---
--- [DEBUG] 2a. Converting PDF (ALL PAGES)... ---
--- [DEBUG] 3. File loaded. Total pages: 2 ---

--- [DEBUG] Processing PAGE 1 / 2 ---
--- [DEBUG] 4. (Page 1) Running YOLO predict... ---
--- [DEBUG] 5. (Page 1) YOLO found 0 objects. ---

--- [DEBUG] Processing PAGE 2 / 2 ---
--- [DEBUG] 4. (Page 2) Running YOLO predict... ---
--- [DEBUG] 5. (Page 2) YOLO found 2 objects. ---
--- [DEBUG] 6. (Page 2) Processing object 1: stamp ---
--- [DEBUG] 6. (Page 2) Processing object 2: stamp ---
--- [DEBUG] 7. All 2 pages processed. ---
--- [DEBUG] 8. Pipeline finished successfully! ---
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cb6897cb2e144196a7.gradio.live


**Секция как мы тренировали модель, подбирали гиперпараметры и тд**


In [ ]:
import cv2
import os
import numpy as np

def make_white_transparent(input_path, output_path):
    img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

    if img is None:
        print(f"Ошибка чтения файла (img is None): {input_path}")
        return

    if len(img.shape) < 3:
        print(f"Изображение в оттенках серого: {input_path}. Конвертируем в RGB.")
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    if img.shape[2] == 3:
        b, g, r = cv2.split(img)
        alpha = np.ones(b.shape, dtype=np.uint8) * 255
        img = cv2.merge((b, g, r, alpha))

    lower_white = np.array([200, 200, 200, 255])
    upper_white = np.array([255, 255, 255, 255])

    if img.shape[2] == 4:
        mask_white = cv2.inRange(img, lower_white, upper_white)
        img[mask_white > 0, 3] = 0
    else:
        print(f"Не удалось определить каналы для прозрачности: {input_path}")
        return

    cv2.imwrite(output_path, img)
    print(f"Обработано: {input_path} -> {output_path}")

def process_recursively(input_root_folder, output_root_folder):
    os.makedirs(output_root_folder, exist_ok=True)

    for root, dirs, files in os.walk(input_root_folder):
        current_subfolder = os.path.relpath(root, input_root_folder)
        output_subfolder = os.path.join(output_root_folder, current_subfolder)
        os.makedirs(output_subfolder, exist_ok=True)

        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, filename)
                output_filename = os.path.splitext(filename)[0] + '.png'
                output_path = os.path.join(output_subfolder, output_filename)

                make_white_transparent(input_path, output_path)

INPUT_FOLDER = '/content/dataset/qr_codes'
OUTPUT_FOLDER = '/content/codes_transparent'

print(f"Начало рекурсивной обработки в: {INPUT_FOLDER}")
process_recursively(INPUT_FOLDER, OUTPUT_FOLDER)
print(f"Обработка завершена. Результаты в: {OUTPUT_FOLDER}")

Начало рекурсивной обработки в: /content/dataset/qr_codes
Изображение в оттенках серого: /content/dataset/qr_codes/qr_0.png. Конвертируем в RGB.
Обработано: /content/dataset/qr_codes/qr_0.png -> /content/codes_transparent/./qr_0.png
Изображение в оттенках серого: /content/dataset/qr_codes/qr_96.png. Конвертируем в RGB.
Обработано: /content/dataset/qr_codes/qr_96.png -> /content/codes_transparent/./qr_96.png
Изображение в оттенках серого: /content/dataset/qr_codes/qr_47.png. Конвертируем в RGB.
Обработано: /content/dataset/qr_codes/qr_47.png -> /content/codes_transparent/./qr_47.png
Изображение в оттенках серого: /content/dataset/qr_codes/qr_58.png. Конвертируем в RGB.
Обработано: /content/dataset/qr_codes/qr_58.png -> /content/codes_transparent/./qr_58.png
Изображение в оттенках серого: /content/dataset/qr_codes/qr_32.png. Конвертируем в RGB.
Обработано: /content/dataset/qr_codes/qr_32.png -> /content/codes_transparent/./qr_32.png
Изображение в оттенках серого: /content/dataset/qr_code

In [ ]:
!yolo segment train model=yolov8s-seg.pt \
    data=/content/synthetic_dataset_V3/data.yaml \
    epochs=50 \
    batch=16 \
    imgsz=640 \
    name=yolo_document_parser

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/synthetic_dataset_V3/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_document_parser, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, persp

In [ ]:
from ultralytics import YOLO,settings
settings.update({'mlflow': True})

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
import random
import glob
import cv2
import numpy as np
from PIL import Image

# --- 1. КОНФИГУРАЦИЯ ---

# Пути к твоим ассетам
BG_DIR = '/content/dataset/background'
STAMP_DIR = '/content/stamps_transparent'
SIGNATURE_DIR = '/content/dataset_transparent'
QR_CODE_DIR = '/content/codes_transparent'

# Куда сохранять готовый датасет
OUTPUT_DIR = '/content/synthetic_dataset_V3'
NUM_IMAGES_TO_GENERATE = 1000  # Сколько картинок создать (1000 для старта)
TRAIN_VAL_SPLIT = 0.9          # 90% в train, 10% в val

# Определение классов для YOLO
CLASS_MAP = {
    'stamp': 0,
    'signature': 1,
    'qrcode': 2
}
# Обратная карта для удобства
ASSET_DIRS = {
    'stamp': STAMP_DIR,
    'signature': SIGNATURE_DIR,
    'qrcode': QR_CODE_DIR
}

# --- 2. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ---

def load_asset_paths():
    """Загружает пути ко всем файлам ассетов."""
    paths = {
        'bg': glob.glob(os.path.join(BG_DIR, '*.*')),
        'stamp': glob.glob(os.path.join(STAMP_DIR, '*.png')),
        'signature': glob.glob(os.path.join(SIGNATURE_DIR, '*.png')),
        'qrcode': glob.glob(os.path.join(QR_CODE_DIR, '*.png'))
    }
    print(f"Найдено фонов: {len(paths['bg'])}")
    print(f"Найдено печатей: {len(paths['stamp'])}")
    print(f"Найдено подписей: {len(paths['signature'])}")
    print(f"Найдено QR-кодов: {len(paths['qrcode'])}")
    return paths

def augment_asset(asset_pil):
    """Применяет легкие аугментации к PNG-ассету (поворот, масштаб, прозрачность)."""

    # 1. Случайный поворот
    angle = random.randint(-15, 15)
    asset_pil = asset_pil.rotate(angle, expand=True, resample=Image.BICUBIC)

    # 2. Случайный масштаб
    scale_factor = random.uniform(0.7, 1.3)
    new_size = (int(asset_pil.width * scale_factor), int(asset_pil.height * scale_factor))
    asset_pil = asset_pil.resize(new_size, resample=Image.LANCZOS)

    # 3. Случайная прозрачность (альфа-канал)
    alpha = asset_pil.getchannel('A')
    new_alpha_data = (np.array(alpha) * random.uniform(0.85, 1.0)).astype(np.uint8)
    new_alpha = Image.fromarray(new_alpha_data)
    asset_pil.putalpha(new_alpha)

    return asset_pil

def get_polygon_from_alpha(asset_pil):
    """
    Извлекает полигон (маску) из альфа-канала PNG-изображения.
    Это самая важная функция для сегментации.
    """
    # Конвертируем PIL в OpenCV (нужен массив numpy)
    asset_np = np.array(asset_pil)

    # Получаем альфа-канал (4-й канал, индекс 3)
    if asset_np.shape[2] != 4:
        raise ValueError("Ассет должен быть PNG с альфа-каналом (RGBA).")

    alpha_channel = asset_np[:, :, 3]

    # Создаем бинарную маску (все, что не полностью прозрачно)
    _, binary_mask = cv2.threshold(alpha_channel, 1, 255, cv2.THRESH_BINARY)

    # Ищем контуры на этой маске
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    # Обычно самый большой контур - это наш объект
    largest_contour = max(contours, key=cv2.contourArea)

    # Упрощаем контур, чтобы в .txt файле было меньше точек
    # epsilon = 0.001 * cv2.arcLength(largest_contour, True)
    # approx_polygon = cv2.approxPolyDP(largest_contour, epsilon, True)

    # Для YOLOv8-seg можно использовать и не упрощенный контур
    # (убрал approxPolyDP для большей точности)

    # Убираем лишнее измерение
    polygon = largest_contour.squeeze(1)
    return polygon

def normalize_polygon(polygon, bg_width, bg_height):
    """Конвертирует абсолютные (пиксели) координаты полигона в
       относительные (0-1), которые требует YOLO."""

    # .astype(float) важно, чтобы при делении получить дробные числа
    polygon_float = polygon.astype(float)

    polygon_float[:, 0] /= bg_width  # Нормализуем X
    polygon_float[:, 1] /= bg_height # Нормализуем Y

    # Ограничиваем значения от 0.0 до 1.0 на всякий случай
    polygon_float = np.clip(polygon_float, 0.0, 1.0)

    return polygon_float

# --- 3. ГЛАВНЫЙ ЦИКЛ ГЕНЕРАЦИИ ---

def main():
    print("Запуск скрипта генерации...")
    asset_paths = load_asset_paths()

    # Создаем папки для датасета
    img_train_dir = os.path.join(OUTPUT_DIR, 'images', 'train')
    lbl_train_dir = os.path.join(OUTPUT_DIR, 'labels', 'train')
    img_val_dir = os.path.join(OUTPUT_DIR, 'images', 'val')
    lbl_val_dir = os.path.join(OUTPUT_DIR, 'labels', 'val')

    os.makedirs(img_train_dir, exist_ok=True)
    os.makedirs(lbl_train_dir, exist_ok=True)
    os.makedirs(img_val_dir, exist_ok=True)
    os.makedirs(lbl_val_dir, exist_ok=True)

    for i in range(NUM_IMAGES_TO_GENERATE):
        print(f"Генерация изображения {i+1}/{NUM_IMAGES_TO_GENERATE}...")

        # 1. Выбираем фон
        bg_path = random.choice(asset_paths['bg'])
        background = Image.open(bg_path).convert("RGBA")
        bg_width, bg_height = background.size

        yolo_labels = [] # Список для хранения строк .txt файла

        # 2. Решаем, сколько каких объектов добавить
        num_stamps = random.randint(1, 2)
        num_signatures = random.randint(1, 2)
        num_qrcodes = random.randint(0, 1) # QR-код может и не быть

        objects_to_add = (
            [('stamp', num_stamps)] +
            [('signature', num_signatures)] +
            [('qrcode', num_qrcodes)]
        )

        # 3. Добавляем объекты на фон
        for class_name, num_to_add in objects_to_add:
            for _ in range(num_to_add):
                try:
                    # Выбираем ассет
                    asset_path = random.choice(asset_paths[class_name])
                    asset_pil = Image.open(asset_path).convert("RGBA")

                    # Аугментируем
                    asset_pil_aug = augment_asset(asset_pil)

                    # Получаем его локальный полигон (маску)
                    local_polygon = get_polygon_from_alpha(asset_pil_aug)
                    if local_polygon is None:
                        print(f"ПРЕДУПРЕЖДЕНИЕ: Не удалось получить полигон для {asset_path}. Пропуск.")
                        continue

                    # Выбираем случайное место для вставки
                    # (с отступом, чтобы не вылезти за края)
                    max_x = bg_width - asset_pil_aug.width - 10
                    max_y = bg_height - asset_pil_aug.height - 10
                    if max_x <= 10 or max_y <= 10:
                        # Ассет слишком большой для этого фона, пропускаем
                        continue

                    paste_x = random.randint(10, max_x)
                    paste_y = random.randint(10, max_y)

                    # Вклеиваем ассет на фон, используя его альфа-канал как маску
                    background.paste(asset_pil_aug, (paste_x, paste_y), mask=asset_pil_aug)

                    # Конвертируем локальный полигон в глобальный (смещаем)
                    global_polygon = local_polygon + [paste_x, paste_y]

                    # Нормализуем для YOLO
                    normalized_polygon = normalize_polygon(global_polygon, bg_width, bg_height)

                    # Форматируем строку для YOLO
                    # <class_id> <x1> <y1> <x2> <y2> ...
                    class_id = CLASS_MAP[class_name]
                    polygon_flat_str = " ".join([f"{coord:.6f}" for coord in normalized_polygon.flatten()])
                    yolo_labels.append(f"{class_id} {polygon_flat_str}")

                except Exception as e:
                    print(f"ОШИБКА при обработке ассета {asset_path}: {e}")
                    continue

        # 4. Сохраняем результат
        if not yolo_labels:
            print("ПРЕДУПРЕЖДЕНИЕ: На изображении не оказалось объектов. Пропуск.")
            continue

        # Решаем, в train или val пойдет картинка
        if random.random() < TRAIN_VAL_SPLIT:
            img_dir = img_train_dir
            lbl_dir = lbl_train_dir
        else:
            img_dir = img_val_dir
            lbl_dir = lbl_val_dir

        # Сохраняем .jpg (конвертируем из RGBA в RGB, т.к. фон мог быть .png)
        img_filename = f"synth_{i:05d}.jpg"
        background.convert("RGB").save(os.path.join(img_dir, img_filename))

        # Сохраняем .txt лейбл
        lbl_filename = f"synth_{i:05d}.txt"
        with open(os.path.join(lbl_dir, lbl_filename), 'w') as f:
            f.write("\n".join(yolo_labels))

    print(f"Готово! {NUM_IMAGES_TO_GENERATE} изображений сгенерировано в папке '{OUTPUT_DIR}'.")

if __name__ == "__main__":
    main()

Запуск скрипта генерации...
Найдено фонов: 65
Найдено печатей: 34
Найдено подписей: 32
Найдено QR-кодов: 100
Генерация изображения 1/1000...
Генерация изображения 2/1000...
Генерация изображения 3/1000...
Генерация изображения 4/1000...
Генерация изображения 5/1000...
Генерация изображения 6/1000...
Генерация изображения 7/1000...
Генерация изображения 8/1000...
Генерация изображения 9/1000...
Генерация изображения 10/1000...
Генерация изображения 11/1000...
Генерация изображения 12/1000...
Генерация изображения 13/1000...
Генерация изображения 14/1000...
Генерация изображения 15/1000...
Генерация изображения 16/1000...
Генерация изображения 17/1000...
Генерация изображения 18/1000...
Генерация изображения 19/1000...
Генерация изображения 20/1000...
Генерация изображения 21/1000...
Генерация изображения 22/1000...
Генерация изображения 23/1000...
Генерация изображения 24/1000...
Генерация изображения 25/1000...
Генерация изображения 26/1000...
Генерация изображения 27/1000...
Генерация

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (226 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!zip -r "xs_dataset.zip" "synthetic_dataset_V3"

In [ ]:
!yolo checks

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.9/112.6 GB disk)

OS                     Linux-6.6.105+-x86_64-with-glibc2.35
Environment            Colab
Python                 3.12.12
Install                pip
Path                   /usr/local/lib/python3.12/dist-packages/ultralytics
RAM                    12.67 GB
Disk                   38.9/112.6 GB
CPU                    Intel Xeon CPU @ 2.20GHz
CPU count              2
GPU                    Tesla T4, 15095MiB
GPU count              1
CUDA                   12.6

numpy                  ✅ 2.0.2>=1.23.0
matplotlib             ✅ 3.10.0>=3.3.0
opencv-python          ✅ 4.12.0.88>=4.6.0
pillow                 ✅ 11.3.0>=7.1.2
pyyaml                 ✅ 6.0.3>=5.3.1
requests               ✅ 2.32.4>=2.23.0
scipy                  ✅ 1.16.3>=1.4.1
torch                  ✅ 2.8.0+cu126>=1.8.0
torch                  ✅ 2.8.0+cu126!=2.4.0,>=1.8.0; sys_platform == "win32"


**Привет это решение созданное за 24 часа ⏰DM[!tg:](https://t.me/fadeinvoid)
[!GitHub](https://github.com/dxrkravehub)**